In [ ]:
pip install requests beautifulsoup4

# Install necessary packages

# Import required libraries
import requests
from bs4 import BeautifulSoup
import re
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt


# Fetch a web page
def fetch_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        return None

# Index words from the page
def index_words(soup):
    index = {}
    words = re.findall(r'\w+', soup.get_text())
    for word in words:
        word = word.lower()
        if word in index:
            index[word] += 1
        else:
            index[word] = 1
    return index

# Remove stop words from the index
def remove_stop_words(index):
    stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at'}
    for stop_word in stop_words:
        if stop_word in index:
            del index[stop_word]
    return index

# Apply stemming to the words in the index
def apply_stemming(index):
    stemmer = PorterStemmer()
    stemmed_index = {}
    for word, count in index.items():
        stemmed_word = stemmer.stem(word)
        if stemmed_word in stemmed_index:
            stemmed_index[stemmed_word] += count
        else:
            stemmed_index[stemmed_word] = count
    return stemmed_index

# Search for a query in the indexed words
def search(index):
    stemmer = PorterStemmer()
    results = {}
    for word in index:
        word = stemmer.stem(word)
        if word in index:
            results[word] = index[word]
    return results

# Function to sort the index from big to small
def sort_index(index):
    sorted_index = sorted(index.items(), key=lambda item: item[1], reverse=True)
    return sorted_index

# Complete search engine function
def search_engine(url):
    soup = fetch_page(url)
    if soup is None:
        return None
    index = index_words(soup)
    index = remove_stop_words(index)
    index = apply_stemming(index)
    results = search(index)
    return results
def plot_top_ten_line(sorted_index):
    words = [item[0] for item in sorted_index]
    counts = [item[1] for item in sorted_index]
    plt.figure(figsize=(10, 6))
    plt.plot(words, counts, marker='o')
    plt.xlabel('Words')
    plt.ylabel('Counts')
    plt.title('Top 10 Words by Frequency')
    plt.grid(True)
    plt.show()

# Example usage
url = 'https://cad.onshape.com/help/Content/Glossary/glossary.htm?tocpath=_____19'

neFet = fetch_page(url)
newIndex = remove_stop_words(index_words(neFet))
indexSroted = sort_index(newIndex)
tenTopword = indexSroted[:10]
print(tenTopword)

!pip install firebase

from firebase import firebase

FBconn = firebase.FirebaseApplication('https://searchengine-79c9e-default-rtdb.firebaseio.com/',None)
def save_to_firebase(sorted_index):
    for i in range(len(sorted_index)):
        index_entry = {
            'Word': sorted_index[i][0],
            'Count': sorted_index[i][1]
        }
        FBconn.post('/Index', index_entry)

save_to_firebase(tenTopword)
result = FBconn.get('Index', None)
print(result)

plot_top_ten(tenTopword)